# Оценка и развертывание моделей временных рядов

##  Цели занятия
- Разобрать ключевые метрики качества моделей временных рядов.
- Научиться строить временную кросс‑валидацию и walk‑forward оценку.
- Сравнить несколько простых моделей прогнозирования на синтетических данных.
- Получить базовое понимание того, как модель можно обернуть в API (FastAPI) и использовать в продакшене.
- Выполнить серию практических заданий от базовых до мини‑проекта.


## 0. Подготовка окружения


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import textwrap

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set_style('whitegrid')

print('✅ Библиотеки загружены')


## 1. Теория: метрики качества прогнозов

Для моделей временных рядов популярны следующие метрики:

- **MAE (Mean Absolute Error)** — средняя абсолютная ошибка.
- **RMSE (Root Mean Squared Error)** — корень из средней квадратичной ошибки.
- **MAPE (Mean Absolute Percentage Error)** — средняя абсолютная процентная ошибка.
- **sMAPE (Symmetric MAPE)** — симметричный вариант, устойчивее к нулям.
- **MASE (Mean Absolute Scaled Error)** — масштабированная ошибка, сравнимая с наивным прогнозом.

Важно помнить, что:
- MAPE и sMAPE чувствительны к значениям, близким к нулю.
- RMSE сильнее штрафует крупные ошибки.
- MASE позволяет сравнивать модели на разных рядах.


In [ ]:
def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def smape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    denom = (np.abs(y_true) + np.abs(y_pred))
    mask = denom != 0
    return 200 * np.mean(np.abs(y_true[mask] - y_pred[mask]) / denom[mask])

def mase(y_true, y_pred, y_train):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_train = np.array(y_train)
    mae_model = mae(y_true, y_pred)
    if len(y_train) < 2:
        return np.nan
    mae_naive = mae(y_train[1:], y_train[:-1])
    if mae_naive == 0:
        return np.nan
    return mae_model / mae_naive

def all_metrics(y_true, y_pred, y_train):
    return {
        'MAE': mae(y_true, y_pred),
        'RMSE': rmse(y_true, y_pred),
        'MAPE': mape(y_true, y_pred),
        'sMAPE': smape(y_true, y_pred),
        'MASE': mase(y_true, y_pred, y_train)
    }

print('✅ Функции метрик определены')


## 2. Генерация синтетических данных

Смоделируем временной ряд с трендом, сезонностью и шумом, а также с несколькими выбросами. Это позволит оценить поведение моделей в чуть более реалистичном сценарии.


In [ ]:
np.random.seed(42)
n = 350
t = np.arange(n)

trend = 0.02 * t
daily_season = 5 * np.sin(2 * np.pi * t / 24)
weekly_season = 2 * np.sin(2 * np.pi * t / (24 * 7))
noise = np.random.normal(0, 1.2, n)

y = trend + daily_season + weekly_season + noise

outliers_idx = [60, 120, 200, 260, 300]
y[outliers_idx] += np.array([10, -8, 12, -10, 7])

train_size = 240
y_train = y[:train_size]
y_test = y[train_size:]

plt.figure(figsize=(14, 5))
plt.plot(y, label='Временной ряд')
plt.axvline(train_size, color='red', linestyle='--', label='Граница train/test')
plt.scatter(outliers_idx, y[outliers_idx], color='orange', s=50, label='Выбросы')
plt.legend()
plt.title('Синтетический временной ряд для оценки моделей')
plt.show()

print(f'Длина ряда: {len(y)}, обучение: {len(y_train)}, тест: {len(y_test)}')


## 3. Простые модели прогнозирования

Реализуем несколько базовых моделей, с которыми удобно сравнивать более сложные методы:

- Среднее по обучающей выборке.
- Наивный прогноз (последнее значение).
- Линейный тренд (drift).
- Скользящее среднее с разными окнами.
- Простое экспоненциальное сглаживание.


In [ ]:
def mean_forecast(train, horizon):
    return np.full(horizon, np.mean(train))

def naive_forecast(train, horizon):
    return np.full(horizon, train[-1])

def drift_forecast(train, horizon):
    if len(train) < 2:
        return naive_forecast(train, horizon)
    n = len(train)
    slope = (train[-1] - train[0]) / (n - 1)
    return train[-1] + slope * np.arange(1, horizon + 1)

def moving_average_forecast(train, horizon, window):
    if len(train) < window:
        window = len(train)
    ma = np.convolve(train, np.ones(window) / window, mode='valid')
    last_ma = ma[-1]
    return np.full(horizon, last_ma)

def simple_exponential_smoothing_forecast(train, horizon, alpha=0.3):
    level = train[0]
    for value in train[1:]:
        level = alpha * value + (1 - alpha) * level
    return np.full(horizon, level)

h = len(y_test)

models = {}
models['Mean'] = mean_forecast(y_train, h)
models['Naive'] = naive_forecast(y_train, h)
models['Drift'] = drift_forecast(y_train, h)
models['MA_7'] = moving_average_forecast(y_train, h, window=7)
models['MA_30'] = moving_average_forecast(y_train, h, window=30)
models['SES_0.3'] = simple_exponential_smoothing_forecast(y_train, h, alpha=0.3)
models['SES_0.7'] = simple_exponential_smoothing_forecast(y_train, h, alpha=0.7)

print('✅ Базовые модели прогнозирования посчитаны')


## 4. Таблица метрик для всех моделей


In [ ]:
rows = []
for name, pred in models.items():
    m = all_metrics(y_test, pred, y_train)
    m['Model'] = name
    rows.append(m)

results_df = pd.DataFrame(rows)
results_df = results_df[['Model', 'MAE', 'RMSE', 'MAPE', 'sMAPE', 'MASE']]
results_df_sorted = results_df.sort_values('RMSE')
results_df_sorted.reset_index(drop=True, inplace=True)
results_df_sorted.round(3)

display(results_df_sorted.round(3))

best_model_name = results_df_sorted.iloc[0]['Model']
print(f'Лучшей по RMSE оказалась модель: {best_model_name}')


## 5. Визуализация прогнозов и ошибок


In [ ]:
plt.figure(figsize=(16, 10))

# График 1: все прогнозы на тесте
plt.subplot(2, 2, 1)
plt.plot(y, label='Истинный ряд', color='black', linewidth=2)
plt.axvline(train_size, color='red', linestyle='--', label='Граница train/test')
for name, pred in models.items():
    plt.plot(range(train_size, n), pred, linestyle='--', label=name, alpha=0.7)
plt.title('Прогнозы разных моделей')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# График 2: heatmap метрик
plt.subplot(2, 2, 2)
metric_matrix = results_df.set_index('Model')[['MAE', 'RMSE', 'MAPE', 'sMAPE']]
sns.heatmap(metric_matrix, annot=True, fmt='.2f', cmap='YlOrRd')
plt.title('Матрица метрик')

# График 3: резидуалы лучшей модели
plt.subplot(2, 2, 3)
best_pred = models[best_model_name]
residuals = y_test - best_pred
plt.plot(residuals, label='Резидуалы')
plt.axhline(0, color='black', linewidth=1)
plt.title(f'Резидуалы лучшей модели ({best_model_name})')
plt.legend()

# График 4: распределение резидуалов
plt.subplot(2, 2, 4)
sns.histplot(residuals, kde=True)
plt.title('Распределение резидуалов')

plt.tight_layout()
plt.show()


## 6. Временная кросс‑валидация (TimeSeriesSplit)

Для временных рядов нельзя перемешивать данные — разбиение train/test должно сохранять порядок. Один из вариантов — `TimeSeriesSplit`, который последовательно расширяет обучающую выборку.


In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
cv_rows = []

for split_idx, (train_idx, val_idx) in enumerate(tscv.split(y_train)):
    y_tr = y_train[train_idx]
    y_val = y_train[val_idx]
    h_cv = len(y_val)
    
    preds_split = {
        'Naive': naive_forecast(y_tr, h_cv),
        'Drift': drift_forecast(y_tr, h_cv),
        'MA_7': moving_average_forecast(y_tr, h_cv, 7),
        'SES_0.3': simple_exponential_smoothing_forecast(y_tr, h_cv, 0.3)
    }
    
    for name, pred in preds_split.items():
        m = all_metrics(y_val, pred, y_tr)
        m['Model'] = name
        m['Split'] = split_idx
        cv_rows.append(m)

cv_df = pd.DataFrame(cv_rows)
cv_summary = cv_df.groupby('Model')[['MAE', 'RMSE', 'MAPE', 'sMAPE', 'MASE']].mean().round(3)
display(cv_summary)
print('\nСредние CV‑ошибки по 5 сплитам отображены выше')


## 7. Walk‑forward валидация (расширенный подход)

В walk‑forward подходе модель последовательно обучается на данных до текущего момента и прогнозирует следующий шаг, затем окно сдвигается вперед. Это приближено к реальному использованию модели.


In [ ]:
def walk_forward_forecast(series, start_train, horizon=1, model_func=naive_forecast):
    series = np.array(series)
    preds = []
    actuals = []
    for t in range(start_train, len(series) - horizon + 1):
        train = series[:t]
        actual = series[t:t + horizon]
        pred = model_func(train, horizon)
        preds.append(pred[-1])
        actuals.append(actual[-1])
    return np.array(actuals), np.array(preds)

wf_start = 100
wf_actual_naive, wf_pred_naive = walk_forward_forecast(y, wf_start, horizon=1, model_func=naive_forecast)
wf_actual_drift, wf_pred_drift = walk_forward_forecast(y, wf_start, horizon=1, model_func=drift_forecast)

print('Walk‑forward Naive MAE:', mae(wf_actual_naive, wf_pred_naive))
print('Walk‑forward Drift MAE:', mae(wf_actual_drift, wf_pred_drift))

plt.figure(figsize=(14, 5))
plt.plot(range(wf_start, wf_start + len(wf_actual_naive)), wf_actual_naive, label='Факт')
plt.plot(range(wf_start, wf_start + len(wf_pred_naive)), wf_pred_naive, label='Naive WF')
plt.plot(range(wf_start, wf_start + len(wf_pred_drift)), wf_pred_drift, label='Drift WF')
plt.title('Walk‑forward прогноз (горизонт 1 шаг)')
plt.legend()
plt.show()


## 8. Идея развёртывания: простая продакшен‑модель и API

В реальном проекте нужно не только обучить модель, но и уметь вызывать её для новых данных. Здесь дадим пример кода, как простую модель можно упаковать и использовать в API.

Ниже будет:
- сохранение конфигурации «модели» в файл,
- пример кода API на FastAPI (как текст, чтобы не запускалось автоматически в ноутбуке).


In [ ]:
import json
import os

# Будем разворачивать лучшую из простых моделей по RMSE
best_model_name = results_df_sorted.iloc[0]['Model']
print('Выбрана модель для условного продакшена:', best_model_name)

prod_config = {
    'model_name': str(best_model_name),
    'train_mean': float(np.mean(y_train)),
    'train_last': float(y_train[-1]),
    'train_first': float(y_train[0]),
    'series_length': int(len(y_train))
}

config_path = 'production_model_config.json'
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(prod_config, f, ensure_ascii=False, indent=2)

print('Конфигурация модели сохранена в', os.path.abspath(config_path))


### 8.1 Пример кода API на FastAPI (как текст)

Код ниже выводится в ячейке как обычный текст, чтобы не мешать исполнению ноутбука. При желании его можно скопировать в отдельный файл `api.py` и запустить.


In [ ]:
api_code = '''\
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
import json

app = FastAPI(title="Time Series Forecast API")

with open("production_model_config.json", "r", encoding="utf-8") as f:
    CONFIG = json.load(f)

class ForecastRequest(BaseModel):
    history: List[float]
    horizon: int = 1

def simple_model_predict(history, horizon):
    model_name = CONFIG["model_name"]
    train_mean = CONFIG["train_mean"]
    train_last = history[-1]
    if model_name == "Mean":
        return [train_mean] * horizon
    elif model_name == "Naive":
        return [train_last] * horizon
    elif model_name == "Drift":
        first = CONFIG["train_first"]
        n = CONFIG["series_length"]
        slope = (train_last - first) / max(n - 1, 1)
        return [train_last + slope * (i + 1) for i in range(horizon)]
    else:
        return [train_last] * horizon

@app.post("/predict")
def predict(req: ForecastRequest):
    if len(req.history) < 5:
        raise HTTPException(status_code=400, detail="Недостаточно точек в history (минимум 5)")
    preds = simple_model_predict(req.history, req.horizon)
    return {"forecast": preds}

# Запуск: uvicorn api:app --reload
'''

print(textwrap.indent(api_code, prefix=''))


## 9. Задания


###  Задание 1 — анализ метрик
- На основе таблицы метрик:
  1. Сравните модели по MAE и RMSE. Совпадает ли ранжирование моделей по этим метрикам?
  2. Посмотрите, как меняется порядок моделей, если сортировать по MAPE или sMAPE.
  3. Сделайте короткую рефлексию о том, какие модели лучше всего подходят для этого ряда и почему.

###  Задание 2 — добавление новых метрик 
- Реализуйте ещё как минимум две метрики, например:
  - Theil’s U,
  - Median Absolute Error,
  - R² (коэффициент детерминации) для прогнозов.
- Добавьте их в общую функцию `all_metrics` и обновите таблицу результатов.
- Визуализируйте результаты на радиальной диаграмме (radar plot) для 3–4 выбранных моделей.

###  Задание 3 — расширенная кросс‑валидация 
- Реализуйте свою версию walk‑forward валидации, которая:
  1. Поддерживает горизонт прогноза `h > 1`.
  2. Вычисляет метрики на каждом шаге и усредняет их по всем окнам.
  3. Сравнивает 3 выбранные вами модели (например, Naive, Drift, MA_30).
- Сформируйте таблицу с результатами и сделайте выводы, совпадают ли оценки с простой train/test‑оценкой.
